### Modules ###

In [27]:
import numpy as np
import helper as h

import operator
from functools import reduce

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline

from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec

from IPython.core.display import display, HTML

### Example text ###

In [28]:
text = '''
Głównym bohaterem jest Gabriel Laymann, nowicjusz klasztoru dominikanów w Mainz. Akcja toczy się w roku 1602 roku gdzie wraz z resztą braci bierze udział w polowaniu na czarownice. Jednak niespodziewane spotkanie swojej przyrodniej siostry Anny Held, osądzanej o wykonywanie czarów, powoduje że Laymann odrzuca regułę klasztoru. Podstępnie zakrada się do biblioteki gdzie studiuje zakazane księgi. Zostaje przyłapany przez swojego nauczyciela brata Jakoba i uwięziony w lochu.
Główny bohater spotyka tam starszego człowieka Lugaida Vandroiy, który przedstawia mu się jako druid (Reach Out For The Light). Spotkany człowiek opowiada Gabrielowi o innym zagrożonym wymiarze – Avantasii. W zamian za pomoc w uratowaniu Avantasii druid obiecuje uratowanie Anny. Razem udaje im się uciec z lochów (Breaking Away) po czym Vandroiy zabiera Gabriela do kamieniołomu, gdzie ukryty jest portal łączący oba światy. Vandroiy używa go by przenieść Gabriela do równoległego świata.
W tym czasie biskup Mainz Johann Adam von Bicken, brat Jakob oraz rządca Falk von Kronberg są w drodze do Rzymu gdzie zamierzają spotkać się z papieżem Clemensem VIII (Glory of Rome). Niosą ze sobą także księgę odkrytą przez Gabriela. Wedle starożytnego zapisu wynika że księga jest ostatnią siódmą częścią pieczęci, która w całości daje właścicielowi absolutną wiedzę gdy tylko dostanie się on do wieży wyznaczającej środek Avantasii.
Gdy Gabriel dostaje się do równoległego świata jest powitany przez dwóch jego mieszkańców elfa Elderana oraz krasnoluda Regrina (Inside). Opowiadają mu o toczącej się wojnie przeciwko siłom zła oraz o planach papieża (Sign Of The Cross). Jeżeli papież użyje pieczęci połączenie między Avantasią a światem ludzi zostanie zamknięte, a oba światy dotkną straszliwe kataklizmy. Gabriel przybywa w momencie gdy Clemens VIII rozmawia z tajemniczym głosem dochodzącym z wieży. Zręcznemu Gabrielowi udaje się skraść pieczęć papieżowi po czym zanosi ją do miasta elfów (The Tower). Zdarzenie to kończy pierwszy album.
Jednak Gabriel nie jest usatysfakcjonowany. Chce dowiedzieć się więcej o świecie Avantasii dlatego Elderane wysyła nowicjusza do drzewa poznania. Tam Gabriel podczas objawienia widzi brata Jakoba który znosił okropny ból w jeziorze ognia (The Final Sacrifice). Elderane opowiada Gabrielowi o złotym kielichu ukrytym w rzymskich katakumbach. Kielich jest więzieniem dla ogromnej ilości torturowanych dusz, artefakt strzeżony jest także przez siejącą postrach bestię. Mimo niepowodzeń delfickich wypraw, Gabriel i Regrin powracają na ziemię by zmierzyć się z bestią. Przyjaciele znajdują kielich i przewracają go co umożliwia ucieczkę duszom. Przebudzona bestia zabija jednak krasnoluda, Gabrielowi udaje się uciec.
Gabriel wraca do Vandroiya, który czekał na niego. Druid spełnia obietnice, zakrada się do więzienia by uwolnić Annę. Jednak znajduje tam „przemienionego” brata Jakoba który także chce uwolnić Annę. Falk von Kronberg nakrywa ich i każe aresztować. Rozpoczęła się walka w której poległ Vandroiy raniony przez Kronberga, który później zostaje uśmiercony przez brata Jakoba. Anna ucieka by ponownie złączyć się z Gabrielem. Podążają wspólnie nieznaną drogą w przyszłość (Into The Unknown).
'''

Source: https://pl.wikipedia.org/wiki/Avantasia

### Split text in to sentences ###

Make a list of raw sentences as well as preprocessed sentences

In [29]:
sentences, raw_sentences = h.prepare_text(text)

In [30]:
sentences

['głównym bohaterem gabriel laymann nowicjusz klasztoru dominikanów mainz',
 'akcja toczy wraz resztą braci bierze udział polowaniu czarownice',
 'niespodziewane spotkanie swojej przyrodniej siostry anny held osądzanej wykonywanie czarów powoduje laymann odrzuca regułę klasztoru',
 'podstępnie zakrada biblioteki studiuje zakazane księgi',
 'zostaje przyłapany swojego nauczyciela brata jakoba uwięziony lochu',
 'główny bohater spotyka starszego człowieka lugaida vandroiy przedstawia druid reach out for the light',
 'spotkany człowiek opowiada gabrielowi innym zagrożonym wymiarze – avantasii',
 'zamian pomoc uratowaniu avantasii druid obiecuje uratowanie anny',
 'razem udaje uciec lochów breaking away czym vandroiy zabiera gabriela kamieniołomu ukryty portal łączący oba światy',
 'vandroiy używa przenieść gabriela równoległego świata',
 'czasie biskup mainz johann adam von bicken brat jakob rządca falk von kronberg drodze rzymu zamierzają spotkać papieżem clemensem glory of rome',
 'nios

### TF-IDF pipeline ###

Prepare TF-IDF pipeline for centroid building

In [31]:
tfidf = Pipeline([
    ('count', CountVectorizer()),
    ('tfidf', TfidfTransformer(norm = None, sublinear_tf = False, smooth_idf = False))
])

In the begging `tfidf.fit_transform(sentences).toarray()` produces a list of shape (n,m) where: 

n - number of sentences, 

m - number of unique tokens across all sentences (vocabulary length)

In [62]:
tfidf.fit_transform(sentences).toarray().shape

(34, 263)

In [63]:
tfidf.fit_transform(sentences).toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 4.52636052, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

### Make centroids ###

Sum up the numbers by column and divide it by the maximum number to obtaint values from 0 to 1 (probability)

In [32]:
centroid_vector_all = tfidf.fit_transform(sentences).toarray().sum(axis = 0)
centroid_vector_all = np.divide(centroid_vector_all, centroid_vector_all.max())

In [7]:
RELEVANT_VECTOR_CUTOFF_RATIO = 0.3 # cutt-of (threshold probability)

Return indices of tokens where the probability is above threshold

In [8]:
relevant_vector_indices = np.where(centroid_vector_all > RELEVANT_VECTOR_CUTOFF_RATIO)[0]

Return tokens of above indices (centroids)

In [9]:
features = tfidf['count'].get_feature_names_out()
word_list = list(np.array(features)[relevant_vector_indices])

In [10]:
word_list # centroids

['anny',
 'annę',
 'avantasii',
 'brata',
 'czym',
 'druid',
 'elderane',
 'falk',
 'gabriel',
 'gabriela',
 'gabrielowi',
 'jakoba',
 'kielich',
 'klasztoru',
 'krasnoluda',
 'kronberg',
 'laymann',
 'mainz',
 'oba',
 'of',
 'opowiada',
 'pieczęci',
 'równoległego',
 'the',
 'uciec',
 'udaje',
 'uwolnić',
 'vandroiy',
 'von',
 'wieży',
 'zakrada',
 'zostaje',
 'świata',
 'światy']

### A lookup model ###

A lokkup model in this case is a Word2Vec dictionary with predefined size and window.

In [66]:
# list of all tokens across all sentences
all_words = [word_tokenize(sent) for sent in sentences]
all_words_flattened = reduce(operator.concat, all_words)

# standard (empty) Word2Vec model
model = Word2Vec(all_words, window=2, size=100, sg = 1, min_count=1)

# populate Word2Vec dictionary
model_lookup = dict()
for word in all_words_flattened:
    model_lookup[word] = model.wv[word]

In [68]:
def make_vector_representation(words: list, model_lookup: dict, model:Word2Vec) -> np.array:
    representation = np.zeros(model.vector_size, dtype='float32')

    for word in words:
        if word in model_lookup.keys():
            representation += model_lookup[word]
    
    representation = np.divide(representation, len(words))

    return representation

In [69]:
centroid_vector = make_vector_representation(word_list, model_lookup, model)

In [70]:
representation = make_vector_representation(all_words_flattened, model_lookup, model)

In [90]:
centroid_vector

array([-5.4278423e-04,  1.5356756e-05, -6.2287884e-04,  3.1565269e-04,
       -1.8956729e-04,  6.8493077e-04,  2.6500228e-04, -7.9654186e-04,
       -6.1839022e-04, -5.9478777e-04,  4.2682426e-04, -2.0681378e-04,
       -4.3802682e-04,  4.9342291e-04, -3.4993899e-04, -5.8126787e-04,
        5.7644491e-05, -1.0343231e-03, -1.8661922e-04,  2.6907458e-04,
        7.6708896e-04,  2.0444910e-04,  3.7846886e-04,  4.5607859e-04,
        6.1902183e-04, -4.5092759e-04, -1.7001628e-04, -2.7660304e-04,
        2.0300491e-04, -3.2841385e-04,  8.1145222e-04,  7.2562706e-04,
       -1.3382996e-05,  2.5918207e-04, -8.5635902e-04, -7.9359516e-04,
        2.9500827e-04, -4.6701389e-04, -4.6107874e-04, -1.3421231e-04,
       -7.4101711e-04, -2.3360019e-04,  2.8383726e-04, -1.6877516e-04,
       -3.5966412e-04,  3.5574427e-04, -5.7928404e-04, -5.1954360e-04,
        7.9186511e-04, -1.1773064e-03,  3.0201700e-04, -1.4514597e-04,
        4.5383986e-05,  1.9800870e-04,  3.3784104e-06, -2.8191030e-04,
      

In [16]:
sent_scores = dict()
for n, sentence in enumerate(sentences):
    words = sentence.split()

    sentence_vector = make_vector_representation(words, model_lookup, model)

    score = h.cos_sim(sentence_vector, centroid_vector)
    sent_scores[n] = [score, sentences[n], sentence_vector, raw_sentences[n]]

sent_scores_sort = sorted(sent_scores.items(), key = lambda item: item[1][0], reverse=True)

In [72]:
sent_scores_sort

[(9,
  [0.47893071236674156,
   'vandroiy używa przenieść gabriela równoległego świata',
   array([ 5.0590700e-04,  5.3545140e-04, -1.6024544e-04, -2.2345332e-03,
           5.4270815e-04,  9.2645868e-04,  5.8445759e-04, -1.4973761e-03,
           4.8913615e-04,  4.3634712e-04,  2.1436273e-03,  3.2741282e-04,
          -5.8291637e-04, -6.4276351e-04,  1.1995398e-03, -2.2222209e-03,
           9.2420407e-04, -2.5537924e-03,  5.0452870e-04, -4.5501770e-04,
           7.3589850e-04, -3.2055145e-04, -7.3786279e-05,  1.4215871e-03,
           4.5613010e-04, -1.4931940e-03, -1.5911181e-03, -6.3168566e-04,
          -3.5464406e-04, -5.0779502e-04,  3.7699344e-03,  3.7468006e-04,
           2.0257093e-04,  1.8093963e-03, -1.7163568e-03, -2.4850431e-03,
           4.0867212e-04, -8.2643220e-04, -1.9107251e-03,  2.0324995e-03,
          -5.0049758e-04,  1.6484701e-04,  1.8194263e-03,  7.7053864e-04,
          -1.1877422e-03,  1.4387607e-03,  1.3605520e-03, -1.3979204e-03,
          -1.0408188e-0

In [73]:
COS_SIM_CUT_OFF = 0.75

In [74]:
for s in sent_scores_sort:
    count = 0
    sentences_summary = []
    #Handle redundancy
    for s in sent_scores_sort:
        if count > 100:
            break
        include_flag = True
        for ps in sentences_summary:
            sim = h.cos_sim(s[1][2], ps[1][2])
            if sim > COS_SIM_CUT_OFF:
                include_flag = False
        if include_flag:
            sentences_summary.append(s)
            count += len(s[1][1].split())
    
    sentences_summary = sorted(sentences_summary, key=lambda el: el[0], reverse=False)

In [75]:
summarized_sents = []
for n, sent in enumerate(sentences_summary):
    summarized_sents.append(sentences_summary[n][1][3])

summarized_sents

['\nGłównym bohaterem jest Gabriel Laymann, nowicjusz klasztoru dominikanów w Mainz',
 'Zostaje przyłapany przez swojego nauczyciela brata Jakoba i uwięziony w lochu',
 'W zamian za pomoc w uratowaniu Avantasii druid obiecuje uratowanie Anny',
 'Razem udaje im się uciec z lochów (Breaking Away) po czym Vandroiy zabiera Gabriela do kamieniołomu, gdzie ukryty jest portal łączący oba światy',
 'Vandroiy używa go by przenieść Gabriela do równoległego świata',
 'Gdy Gabriel dostaje się do równoległego świata jest powitany przez dwóch jego mieszkańców elfa Elderana oraz krasnoluda Regrina (Inside)',
 'Zręcznemu Gabrielowi udaje się skraść pieczęć papieżowi po czym zanosi ją do miasta elfów (The Tower)',
 'Tam Gabriel podczas objawienia widzi brata Jakoba który znosił okropny ból w jeziorze ognia (The Final Sacrifice)',
 'Jednak znajduje tam „przemienionego” brata Jakoba który także chce uwolnić Annę',
 'Falk von Kronberg nakrywa ich i każe aresztować',
 'Rozpoczęła się walka w której poległ 

In [76]:
def display_highlights(raw_sentences, summarized_sents):
    final = []
    for sent in raw_sentences:
        if sent not in summarized_sents:
            final += sent
        else:
            final += f'<span style="background-color:rgba(255,215,0,0.3);"> {sent} </span>'
    display(HTML(''.join([elem for elem in final])))

In [77]:
display_highlights(raw_sentences, summarized_sents)